# 階層式索引

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])
data

a  1    0.960003
   2    1.343741
   3   -1.605240
b  1   -0.864458
   3   -1.223799
c  1   -0.693037
   2   -0.303235
d  2   -0.687881
   3    0.778021
dtype: float64

In [ ]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [ ]:
data['b']

1   -0.864458
3   -1.223799
dtype: float64

In [ ]:
data['b':'c']

b  1   -0.864458
   3   -1.223799
c  1   -0.693037
   2   -0.303235
dtype: float64

In [ ]:
data.loc[['b','d']]

b  1   -0.864458
   3   -1.223799
d  2   -0.687881
   3    0.778021
dtype: float64

In [ ]:
data.loc[:,2]#內層資料

a    1.343741
c   -0.303235
d   -0.687881
dtype: float64

In [ ]:
data.unstack()#重塑資料DataFrame類似r table()

,1,2,3
a,0.960003,1.343741,-1.605240
b,-0.864458,NaN,-1.223799
c,-0.693037,-0.303235,NaN
d,NaN,-0.687881,0.778021


In [ ]:
data.unstack().stack()#堆疊單一不給定列型式

a  1    0.960003
   2    1.343741
   3   -1.605240
b  1   -0.864458
   3   -1.223799
c  1   -0.693037
   2   -0.303235
d  2   -0.687881
   3    0.778021
dtype: float64

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[['a','a','b','b'],[1,2,1,2]],
                     columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [ ]:
frame.index.names = ['key1','key2']
frame.columns.names = ['state','color']
frame#類別資料的時後或許會比較好用

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

## 重排階層及依階層排序值

In [ ]:
frame.swaplevel('key1','key2')#重新排序依照key

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [ ]:
frame.sort_index(level=0)#依照指定層index字典排序

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 指定階層統計資訊

In [ ]:
frame.sum(level='key2')

<ipython-input-31-f3eb061baf48>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [ ]:
frame.sum(level='color',axis=1)

<ipython-input-32-84f67c48058e>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color',axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

##用DataFrame的欄當index



In [ ]:
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two','two','two','two'],'d':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [ ]:
frame2 = frame.set_index(['c','d'])#捨棄欄名稱,變成層
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [ ]:
frame.set_index(['c','d'],drop=False)#留下欄位資料drop

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [ ]:
frame2.reset_index()#set_index相反 變成單純欄資料

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 合併資料(join)

## DataFrame資料庫的join動作

* how 選項中不同的join選項

| 🐦選項 | 😸行為 |
| ------------- |:-------------:|
|'inner'|只用兩邊表格中都有的key|
|'left'|使用左方表格中的所有key|
|'right'|使用右方表格中的所有key|
|'outer'|只要兩邊表格中有出的key都使用|

* merge 的參數

| 🐦參數 | 😸描述 |
| ------------- |:-------------:|
|left|以左邊DataFrame準進行合併|
|right|以右邊DataFrame準進行合併|
|how|可指定'inner'、'outer、'left和'right;預設是'inner'|
|on|指定合併用的key欄位,該欄位必须在兩邊的DataFrame物件中都存在·如未指定此項,也沒有指定其他的合併key的話,用左右兩邊都有出現的相同欄名稱當作合併key|
|left on|以左方DataFrame中的欄位當做合併的key(若是MultiIndex,則為多層key)|
|right_on|以右方DataFrame中的欄位當做合併的key(若是MultiIndex,則為多層key)|
|left index|以左方DataFrame中的index用來作合併的key(若是Multilndex為多層key,則為多層key)|
|right_index|以右方DataFrame中的index用來作合併的key(若是Multilndex為多層key,則為多層key)|
|sort|以合併的key為準,將合併後的資料依字典順序排序;預設為True(若是碰到大型資料集合,關閉這個選項可得到更好的效率)|
|suffixes|指定字串值的tuple,加在重複的欄位名稱後面;預設是('_x','_y')(例如:如果兩邊DataFrame物件有'data'這個欄位,那產結果變成'data_x'和'data_y')|
|copy|如果設定為False,在碰到例外情況時不複製資料到輸出資料結構中;預設是不管怎標都複製|
|indicator|加一個指出列來源的欄;欄中的值會反應列是從哪裡來的,分作'left_on'、'right_only'和'both'三種|

In [ ]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','c'],'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,c,6


In [ ]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [ ]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0


In [ ]:
pd.merge(df1,df2,on='key')#同意思,以兩個欄都有的key作join

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0


In [ ]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})
print(df3)
print(df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


In [ ]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')#分別指定但會自動擴充,似full_join

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [ ]:
pd.merge(df1,df2,how='outer')#兩邊都有的都加但會以NA代替另一邊沒有的

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,a,2.0,0.0
3,a,4.0,0.0
4,a,5.0,0.0
5,c,3.0,NaN
6,c,6.0,NaN
7,d,NaN,2.0


In [ ]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'],'data1':range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'],'data2':range(5)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


In [ ]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [ ]:
pd.merge(df1,df2,on='key',how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [ ]:
left = pd.DataFrame({'key1':['foo','foo','bar'],'key2':['one','two','one'],'lval':[1,2,3]})
right = pd.DataFrame({'key1':['foo','foo','bar','bar'],'key2':['one','one','one','two'],'lval':[4,5,6,7]})
print(left)
print(right)

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  lval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [ ]:
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval_x,lval_y
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [ ]:
pd.merge(left,right,on='key1')

,key1,key2_x,lval_x,key2_y,lval_y
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [ ]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))#suffix點綴備註

,key1,key2_left,lval_left,key2_right,lval_right
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## 依index作合併

In [ ]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [ ]:
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [ ]:
pd.merge(left1,right1,left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [ ]:
lefth = pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                      'key2':[2000,2001,2002,2001,2002],
                      'data':np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                      index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                             [2001,2000,2000,2000,2001,2002]],
                      columns=['event1','event2'])
print(lefth)
print(righth)

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [ ]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [ ]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [ ]:
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],
                     index=['a','c','e'],
                     columns=['Ohio','Nevada'])
right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13.,14.]],
                      index=['b','c','d','e'],
                      columns=['Missouri','Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [ ]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)#類似full_join

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [ ]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [ ]:
left1.join(right1,on='key')#key對應合併位置

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [ ]:
another = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],
                       index=['a','c','e','f'],
                       columns=['New York','Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [ ]:
left2.join([right2,another])#inner_join

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [ ]:
left2.join([right2,another],how='outer')#full_join

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


## 沿軸做連接

* pandas=1.0.0 開始不再支持join_axes
```
pd.concat([s1,s4],axis=1,join='inner',join_axes=[['a','c','b','e']])
```
[解決方法連結](https://www.cnblogs.com/bigtreei/p/14160287.html)

* concat函式參數

| 🐦參數 | 😸描述 |
| ------------- |:-------------:|
|objs|要被連接物件的dict或list;這是唯一的必要參數|
|axis|沿哪個軸作連接;預設為0(沿列)|
|join|可以設定為'inner'或'outer'(預設為'outer');設定其他軸上的index是要使用交集(inner)或是聯集(outer)
|join_axes|進行聯集或交集動作時,指定使用其他的n-1軸作為index|
|keys|用值代表被連接的物件,在連接的軸上最高階層的index;可以是內含多值的list(如果levelst傳的是多層的陣列的話)|
|levels|指定用來建構階層式index一層的值,如果有傳keys話,就變成建構多層用的值|
|names|如果keys及/或levels有傳值的話,這項是新建出的階層式index的名稱|
|verify_integrity|檢查連接物件的新軸中index標籤有沒有重複,有的話就丟出例外;預設為False,即允許重複|
|ignore_index|在沿指定axis做接合時,不要保留原index,用range(total_length)產生的新值作為index|

In [ ]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [ ]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [ ]:
s1 = pd.Series([0,1],index=['a','b'])
s2 = pd.Series([2,3,4],index=['c','d','e'])
s3 = pd.Series([5,6],index=['f','g'])
pd.concat([s1,s3,s3])#list中船入concat可以將值索引接合起來

a    0
b    1
f    5
g    6
f    5
g    6
dtype: int64

In [ ]:
pd.concat([s1,s2,s3],axis=1)#縱向合併,會填值NA

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [ ]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [ ]:
pd.concat([s1,s4],axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [ ]:
pd.concat([s1,s4],axis=1,join='inner')

,0,1
a,0,0
b,1,1


In [ ]:
result = pd.concat([s1,s2,s3],keys=['one','two','three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [ ]:
result.unstack()#table

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [ ]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])
df2 = pd.DataFrame(5+np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])
print(df1)
print(df2)

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


In [ ]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [ ]:
pd.concat({'level1':df1,'level2':df2},axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [ ]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'],
          names=['upper','lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [ ]:
df1 = pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
df2 = pd.DataFrame(np.random.randn(2,3),columns=['b','d','a'])
print(df1)
print(df2)

          a         b         c         d
0  0.707855  1.259557 -1.033017  0.753993
1 -0.401339 -0.193824 -0.279490 -0.181004
2  0.758889  1.806061 -0.455959 -1.686905
          b         d         a
0  0.953034 -0.809448  0.064685
1  1.226910 -0.464439 -1.563810


In [ ]:
pd.concat([df1,df2],ignore_index=True)#疊加上不相關資料ignore_index=True這很有用

,a,b,c,d
0,0.707855,1.259557,-1.033017,0.753993
1,-0.401339,-0.193824,-0.279490,-0.181004
2,0.758889,1.806061,-0.455959,-1.686905
3,0.064685,0.953034,NaN,-0.809448
4,-1.563810,1.226910,NaN,-0.464439


## 合併有重複的資料

In [ ]:
a = pd.Series([np.nan,2.5,0.0,3.5,4.5,np.nan],
              index=['f','e','d','c','b','a'])
b = pd.Series([0.,np.nan,2.,np.nan,np.nan,5.],
              index=['a','b','c','d','e','f'])
print(a)
print(b)

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64
a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64


In [ ]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [ ]:
df1 = pd.DataFrame({'a':[1.,np.nan,5,np.nan],
                    'b':[np.nan,2.,np.nan,6.],
                    'c':range(2,18,4)})
df2 = pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],
                    'b':[np.nan,3.,4.,6.,8.]})
print(df1)
print(df2)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14
     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


In [ ]:
df1.combine_first(df2)#比對修復自身的遺失值

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 重塑和旋轉

## 用階層式index進行重塑

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                    index=pd.Index(['Ohio','Colorado'],name='state'),
                    columns=pd.Index(['one','two','three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [ ]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [ ]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [ ]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [ ]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [ ]:
s1 = pd.Series([0,1,2,3],index=['a','b','c','d'])
s2 = pd.Series([4,5,6],index=['c','d','e'])
data2 = pd.concat([s1,s2],keys=['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [ ]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [ ]:
data2.unstack().stack()#互轉消除NA

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [ ]:
data2.unstack().stack(dropna=False)#有na

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [ ]:
df = pd.DataFrame({'left':result,'right':result+5},
                  columns=pd.Index(['left','right'],name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [ ]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

## 長格式轉成寬格式

In [ ]:
#Colab資料夾設定方法
#!pip install google.colab #如未安裝取消註解後執行
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/python')
os.listdir() #確認目錄內容

['Python course 02-常數與變數_exercise_answer.pptx',
 'Python course 03-運算式_exercise.pptx',
 '無標題文件.gdoc',
 'Python course 04-流程控制- 判斷_exercises.pptx',
 'Python練習.zip',
 'dataset',
 'ipython_script_test.ipynb',
 'some_module.ipynb',
 'tmp.txt',
 'tmp1.txt',
 'tmp2.txt',
 'ex1.csv',
 'ex2.csv',
 'csv_mindex.csv',
 'ex3.txt',
 'ex4.csv',
 'ex5.csv',
 'tseries.csv',
 'ex7.csv',
 'example.json',
 'banklist.html',
 'frame_pickle',
 'my_data.h5',
 'ex1.xlsx',
 'ex2.xlsx',
 'mydata.sqlite',
 'mytdata.h5']

In [ ]:
data = pd.read_table('dataset/macrodata.txt',sep=',')
data = pd.DataFrame(data)
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [ ]:
data.columns

Index(['year', 'quarter', 'realgdp', 'realcons', 'realinv', 'realgovt',
       'realdpi', 'cpi', 'm1', 'tbilrate', 'unemp', 'pop', 'infl', 'realint'],
      dtype='object')

In [ ]:
data.year

0      1959
1      1959
2      1959
3      1959
4      1960
       ... 
198    2008
199    2008
200    2009
201    2009
202    2009
Name: year, Length: 203, dtype: int64

In [ ]:
periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')

In [ ]:
columns = pd.Index(['realgdp','infl','unemp'],name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D','end')
ldata = data.stack().reset_index().rename(columns={0:'value'})

In [ ]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [ ]:
##上面方法超麻煩脫褲子放屁
pivoted = ldata.pivot('date','item','value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [ ]:
ldata['value2']=np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.445208
1,1959-03-31 23:59:59.999999999,infl,0.000,0.384229
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.975215
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.631277
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.152369
5,1959-06-30 23:59:59.999999999,unemp,5.100,1.497801
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.249831
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.656712
8,1959-09-30 23:59:59.999999999,unemp,5.300,-1.368354
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-2.353320


In [ ]:
pivoted = ldata.pivot('date','item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.384229 -0.445208   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.152369  0.631277   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.656712 -0.249831   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.720093 -2.353320   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.879816 -1.149570   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.975215  
1959-06-30 23:59:59.999999999  1.497801  
1959-09-30 23:59:59.999999999 -1.368354  
1959-12-31 23:59:59.999999999  0.909697  
1960-03-31 23:59:59.999999999 -0.172522

In [ ]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [ ]:
unstacked = ldata.set_index(['date','item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.384229 -0.445208   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.152369  0.631277   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.656712 -0.249831   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.720093 -2.353320   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.879816 -1.149570   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2  0.109132 -0.771919   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  0.208868 -1.515913   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.975215  
1959-06-30 23:59:59.999999999  1.497801  
1959-09-30 23:59:59.999999999 -1.368354  
1959-12-31 23:59:59.999999999  0.909697  
1960-03-31 23:59:59.999999999 -0.172522  
1960-06-30 23:59:59.999999999  0.534364  
1960-09-30 23:59:59.999999999  0.910165

## 寬格式轉成長格式

In [ ]:
df = pd.DataFrame({'key':['foo','bar','baz'],
                   'A':[1,2,3],
                   'B':[4,5,6],
                   'C':[7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [ ]:
melted = pd.melt(df,['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [ ]:
reshaped = melted.pivot('key','variable','value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [ ]:
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [ ]:
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [ ]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
